# Document loading

### Setup

Import the existing weaviate instance. Remove any existing schemas before we start.

In [1]:
from pathlib import Path

from manifesto_qa.app import vector_db, WEAVIATE_INDEX_NAME
from manifesto_qa.client import read_all_objects
from manifesto_qa.document_loader import load_and_split_pdf, add_documents_to_store

/Users/longbe01/Documents/projects/llm-rag/venv-llm-rag/lib/python3.10/site-packages/weaviate/warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(
/Users/longbe01/Documents/projects/llm-rag/venv-llm-rag/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_o

In [17]:
vector_db.delete_all_schemas()
vector_db.reset_manifesto_schema()

In [3]:
weaviate_instance = vector_db.instance
weaviate_client = vector_db.client

### Document loading and splitting

Load each of the manifesto PDF documents and split them into chunks. Then, add them to the vector database (using OpenAI's text embeddings model to generate embeddings).

In [19]:
data_dir = Path("/Users/longbe01/Documents/projects/llm-rag/data")
data_dir.as_posix()

'/Users/longbe01/Documents/projects/llm-rag/data'

In [20]:
for file in data_dir.glob("*.pdf"):

    pdf_splits = load_and_split_pdf(file) 
    print(f"Split file {file.name} into {len(pdf_splits)} chunks.")

    docs_added = add_documents_to_store(weaviate_instance, pdf_splits)
    print(f"Added {len(docs_added)} chunks to vector db for file {file.name}.\n")


Split file Plaid_Cymru_Maniffesto_2024_ENGLISH.pdf into 166 chunks.
Added 166 chunks to vector db for file Plaid_Cymru_Maniffesto_2024_ENGLISH.pdf.

Split file Green-Party-2024-General-Election-Manifesto-Long-version_imprint.pdf into 166 chunks.
Added 166 chunks to vector db for file Green-Party-2024-General-Election-Manifesto-Long-version_imprint.pdf.

Split file Change-Labour-Party-Manifesto-2024-large-print.pdf into 252 chunks.
Added 252 chunks to vector db for file Change-Labour-Party-Manifesto-2024-large-print.pdf.

Split file 2024-06-20b-SNP-General-Election-Manifesto-2024_interactive.pdf into 76 chunks.
Added 76 chunks to vector db for file 2024-06-20b-SNP-General-Election-Manifesto-2024_interactive.pdf.

Split file Reform_UK_Contract_With_The_People.pdf into 73 chunks.
Added 73 chunks to vector db for file Reform_UK_Contract_With_The_People.pdf.

Split file Conservative-Manifesto-GE2024.pdf into 235 chunks.
Added 235 chunks to vector db for file Conservative-Manifesto-GE2024.pd

### View the database schema

Understand how the documents have been loaded (and how this relates to creating collections directly using the weaviate API). 

Sense check the documents have all been loaded correctly, and view an example record.

In [4]:
all_records = read_all_objects(weaviate_client, WEAVIATE_INDEX_NAME, 20)

print(f"There are {len(all_records)} objects in the vector database \n")
print("Example object: \n ", all_records[0])

There are 1197 objects in the vector database 

Example object: 
  {'_additional': {'id': '00299358-dd30-419b-ac42-eeca02aef098'}, 'page': 40, 'source': '/Users/longbe01/Documents/projects/llm-rag/data/For_a_Fair_Deal_-_Liberal_Democrat_Manifesto_2024.pdf', 'text': '•  Increase school and college funding per pupil above the rate of inflation every year, and end the scandal of crumbling school and college buildings by investing in new buildings and clearing the backlog of repairs.\n•  Introduce a ‘Tutoring Guarantee’ for every disadvantaged pupil who needs extra support.\n•  Invest in high-quality early years education and close the attainment gap by giving disadvantaged children aged three and four an extra five free hours a week and tripling the Early Years Pupil Premium to £1,000 a year.\n•  Reinstate maintenance grants for disadvantaged students immediately to make sure that living costs are not a barrier to studying at university.'}


In [23]:
weaviate_client.schema.get()['classes'][0]["properties"]

[{'dataType': ['text'],
  'description': 'The document content chunked',
  'indexFilterable': True,
  'indexSearchable': True,
  'moduleConfig': {'text2vec-openai': {'dimensions': 1536,
    'model': 'text-embedding-3-small',
    'skip': False,
    'tokenization': 'lowercase',
    'type': 'text',
    'vectorizePropertyName': True}},
  'name': 'text',
  'tokenization': 'word'},
 {'dataType': ['text'],
  'description': 'The source document (PDF)',
  'indexFilterable': True,
  'indexSearchable': True,
  'moduleConfig': {'text2vec-openai': {'skip': False,
    'tokenization': 'whitespace',
    'vectorizePropertyName': False}},
  'name': 'source',
  'tokenization': 'word'},
 {'dataType': ['number'],
  'description': 'Page number',
  'indexFilterable': True,
  'indexSearchable': False,
  'moduleConfig': {'text2vec-openai': {'skip': False,
    'tokenization': 'whitespace',
    'vectorizePropertyName': False}},
  'name': 'page'}]

In [24]:
weaviate_client.schema.get()['classes'][0]

{'class': 'ManifestoQa',
 'description': 'Index storing political party GE 2024 manifesto documents.',
 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
  'cleanupIntervalSeconds': 60,
  'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
 'moduleConfig': {'generative-openai': {'model': 'gpt-3.5-turbo'},
  'text2vec-openai': {'baseURL': 'https://api.openai.com',
   'model': 'ada',
   'vectorizeClassName': True}},
 'multiTenancyConfig': {'autoTenantCreation': False, 'enabled': False},
 'properties': [{'dataType': ['text'],
   'description': 'The document content chunked',
   'indexFilterable': True,
   'indexSearchable': True,
   'moduleConfig': {'text2vec-openai': {'dimensions': 1536,
     'model': 'text-embedding-3-small',
     'skip': False,
     'tokenization': 'lowercase',
     'type': 'text',
     'vectorizePropertyName': True}},
   'name': 'text',
   'tokenization': 'word'},
  {'dataType': ['text'],
   'description': 'The source document (PDF)',
   'indexF

In [25]:
weaviate_client.schema.get()

{'classes': [{'class': 'ManifestoQa',
   'description': 'Index storing political party GE 2024 manifesto documents.',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'moduleConfig': {'generative-openai': {'model': 'gpt-3.5-turbo'},
    'text2vec-openai': {'baseURL': 'https://api.openai.com',
     'model': 'ada',
     'vectorizeClassName': True}},
   'multiTenancyConfig': {'autoTenantCreation': False, 'enabled': False},
   'properties': [{'dataType': ['text'],
     'description': 'The document content chunked',
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-openai': {'dimensions': 1536,
       'model': 'text-embedding-3-small',
       'skip': False,
       'tokenization': 'lowercase',
       'type': 'text',
       'vectorizePropertyName': True}},
     'name': 'text',
     'tokenization': 'word'},
    {'dataType': ['text'],
